## 音樂推薦

In [ ]:
import os
from flask import Flask, render_template, request, redirect, url_for, flash
import librosa
from keras.models import load_model
import keras
import numpy as np
from sklearn.metrics import classification_report
import librosa.display
import requests
import cv2

In [ ]:
# 配置Flask應用程序
app = Flask(__name__)
UPLOAD_FOLDER = 'uploads/'
ALLOWED_EXTENSIONS = {'mp3', 'wav'}
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

In [ ]:
# 檢查文件是否是允许的音樂格式
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

In [ ]:
# 主要的音樂上傳和處理路由
@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        if 'file' not in request.files:
            flash('No file part')
            return redirect(request.url)
        file = request.files['file']
        if file.filename == '':
            flash('No selected file')
            return redirect(request.url)
        if file and allowed_file(file.filename):
            filepath = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
            file.save(filepath)

            predicted_genre = predict_genre(filepath)
            recommended_links = get_youtube_links(predicted_genre)
            
            return render_template('result.html', genre=predicted_genre, links=recommended_links)
    return render_template('upload.html')

In [ ]:
# 2. 加载預先訓練的模型
import pickle

with open('history_500_epoch_tr_GRU.pkl', 'rb') as file:
    model = pickle.load(file)

def predict_genre(filepath):
    x, sr = librosa.load(filepath)
    X = librosa.stft(x)
    Xdb = librosa.amplitude_to_db(abs(X))

    # 3. 將音頻轉換為模型所需的輸入格式
    # Assuming that you have a function 'convert_to_model_input' 
    # that converts the Xdb to a shape that your model expects
    input_data = convert_to_model_input(Xdb)

    # 4. 使用模型進行預測
    predictions = model.predict(input_data)
    genre = decode_predictions(predictions)  # assuming you have a function to decode the predictions

    return genre

def convert_to_model_input(Xdb):
    # 这里是一个示例，您需要根据实际的模型输入要求进行更改
    return np.expand_dims(Xdb, axis=0)  # 假设模型需要一个四维输入

# def convert_to_model_input(x, sr):
    # 根据您的模型输入需求修改此函数
#    mfccs = librosa.feature.mfcc(x, sr=sr)
    # ... [其他可能的特征提取和预处理]
    
    # 适应模型的输入形状
#    input_data = np.expand_dims(mfccs, axis=0)
    
#    return input_data

def decode_predictions(predictions):
    genres = ['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock']  # 音乐类别列表
    predicted_index = np.argmax(predictions)
    return genres[predicted_index]



In [ ]:
# 根據音樂類型返回推薦的YouTube連結
def get_youtube_links(genre, max_results=5):
    API_KEY = 'YOUR_YOUTUBE_API_KEY'  # You need to get this from the Google Cloud Console.
    YOUTUBE_SEARCH_URL = 'https://www.googleapis.com/youtube/v3/search'

    params = {
        'part': 'snippet',
        'q': genre + ' music',
        'type': 'video',
        'key': API_KEY,
        'maxResults': max_results
    }

    response = requests.get(YOUTUBE_SEARCH_URL, params=params)
    data = response.json()
    video_links = ['https://www.youtube.com/watch?v=' + item['id']['videoId'] for item in data['items']]
    
    return video_links
